# 多标签竞赛分享

#### 由于后期工作会用到多标签分类，刚好又看到了这个竞赛就参加了。分享下过程的一些想法和思路，不确定都是对的，希望各路大佬指导。

#### backbone model：  
&#8195;&#8195;&#8195;&#8195;inceptionresnetv2，  
&#8195;&#8195;&#8195;&#8195;当然也尝试了resnet系列等，  
&#8195;&#8195;&#8195;&#8195;考虑过用1x1的convnet代替fc，但是1x1主要是为了节省模型，所以就算了，在fc的时候应该和很多同学一样感觉features的dim相对于标签数量有点小，1、想增加features的维度，尝试增加了一个模型，concat features，效果一般，考虑到实用也不大就没再继续研究。2、fc分成两个部分，仔细思考了下在features一样的情况下，其实多此一举，除非是不同的feature extraction，或者是两部份分开训练。但考虑实际应用都没再尝试。
                

#### Data augmentation:  
&#8195;&#8195;&#8195;&#8195;RandomRotation(20),  
&#8195;&#8195;&#8195;&#8195;ColorJitter(0.3,0.3,0.3,0.1),  
&#8195;&#8195;&#8195;&#8195;RandomHorizontalFlip(),  
&#8195;&#8195;&#8195;&#8195;RandomVerticalFlip(),  
&#8195;&#8195;&#8195;&#8195;尝试了增加高斯噪声效果不太理想，并没继续研究。

主要尝试可能是再loss上了，直接上代码(基于pytorch)：

In [6]:
import torch
import torch.nn as nn
import numpy as np
class BCEFDiceLoss(nn.Module):
    def __init__(self,
                 bce_weight=3,
                 dice_weight=1):
        super().__init__()

        self.dice_weight = dice_weight
        self.bce_weight = bce_weight
        self.class_weight = torch.FloatTensor([1,10])


    def forward(self, outputs, targets):
        #标签为1权重设置为10，考虑过安装标签数量来设置结果和直接把1设为10差不多，这个更加直接。
        weight = self.class_weight[targets.long()]
        weights = weight.to(targets.device)
        #二分类用的交叉熵：
        if not (targets.size() == outputs.size()):
            raise ValueError(
                "Target size ({}) must be the same as input size ({})".format(targets.size(), outputs.size()))

        max_val = (-outputs).clamp(min=0)
        loss = outputs - outputs * targets + max_val + ((-max_val).exp() + (-outputs - max_val).exp()).log()
        loss = loss*weights
        loss = loss.mean()*self.bce_weight
        #参照fmeasure：
        eps = 1e-10
        dice_target = (targets == 1).float()
        dice_output = F.sigmoid(outputs)
        intersection = (dice_output * dice_target).sum()
        union = dice_output.sum() + dice_target.sum() + eps
        loss += (1 - torch.log(2 * intersection / union)) * self.dice_weight
        return loss

时间有限就这样了，有错请大佬指教。